# Import Required Libraries

In [1]:
import re
import unicodedata
from collections import defaultdict, Counter
import networkx as nx
from itertools import combinations
import difflib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import jdatetime

### Load Dataset

In [2]:
df = pd.read_csv('reviews.csv')

### Basic Cleaning

In [3]:
"""
During EDA, we determined that nearly all missing values were due to: 
survey images, professor suggestion polls, or various single messages 
(like pinned posts, guides, or initial channel messages).
We may revisit EDA-related code later to extract any potentially useful information from these if possible.
For now, we removing them as we did during the EDA phase.
"""

# Just 2 Special Cases
df.loc[df['message_id'] == 816, ['department', 'comment_text', 'parse_error']] = [
    'معارف', 
    'خییلللللی استاد محشرین از دسشون ندین ۴ نمره اضافی دارن برای نهاد که جواباش پیدا میشه😁 امتحاناشونم میانترم کپی ترم پیش و پایانترم از یه فایل ۲۰ صفحه ای بود',
    None
]

df.loc[df['message_id'] == 252, 'comment_text'] = np.nan

df = df.dropna(subset=['term'])

In [4]:
print("Total reviews:", len(df))
print("Unique professors:", df["professor_name_raw"].nunique())
print("Departments:", df["department"].nunique())

Total reviews: 4276
Unique professors: 1296
Departments: 18


In [5]:
df['comment_text'].isna().sum()

np.int64(16)

In [6]:
df.loc[df['department'] == 'شیمی', 'department'] = 'مهندسی_شیمی'

### Term Normalization

In [7]:
temp_grouped = {
 ('قبل از سال 94',): 'before_94',

 ('بهمن 94',): 'second_1394',

 ('تابستان 95',): 'summer_1395',
 
 ('مهر 96',): 'first_1396',
 ('تابستان 96',): 'summer_1396',
 
 ('مهر 97',): 'first_1397',
 ('بهمن 97',): 'second_1397',
 
 ('مهر 98',): 'first_1398',
 ('بهمن 98',): 'second_1398',
 
 ('مهر 99',): 'first_1399',
 ('بهمن 99', 'بهمن ۹۹'): 'second_1399',
 ('تابستان 99',): 'summer_1399',
 
 ('مهر 00', 'مهر 1400', 'مهر ۰۰', 'مهر ..', 'مهر ١٤٠٠', 'مهر ۱۴۰۰', 'مهر۱۴۰۰', 'پاییز ۱۴۰۰'): 'first_1400',
 ('4002', 'بهمن 1400', 'بهمن ۱۴۰۰'): 'second_1400',
 ('تابستان 1400', 'تابستان۱۴۰۰'): 'summer_1400',
 ('بهمن 99 ،مهر 1400', 'مهر و بهمن 1400', 'مهر و بهمن ۱۴۰۰', 'هم ترم مهر ۱۴۰۰ و ترم بهمن ۱۴۰۰'): 'both_1400',
 
 ('14011', 'ترم های 4001 - 4002 - 4011', 'مهر 1401', 'مهر ۴۰۱', 'پاییز ۱۴۰۱'): 'first_1401',
 ('بهار 1402', 'بهار ۱۴۰۲', 'بهار ۴۰۲', 'بهمن 1401', 'بهمن ۴۰۱', 'فرودین ۱۴۰۲'): 'second_1401',
 ('تابستان 1401',): 'summer_1401',
 ('مهر 1401 و بهمن 1402',): 'both_1401',

 ('1402 پاییز', 'مهر 1402', 'مهر 402', 'مهر ۱۴۰۲', 'مهر ۴۰۲', 'مهر۱۴۰۲', 'مهر۴۰۲', 'مهر 1402 - بهمن 1402', 'مهر ۱۴۰۲-بهمن ۱۴۰۲', 'بهمن۱۴۰۱_مهر۱۴۰۲', 'پاییز  ۱۴۰۲', 'پاییز 1402', 'پاییز 402', 'پاییز ۱۴۰۲', '۱۴۰۲ مهر', '۱۴۰۲-۱'): 'first_1402',
 ('4022', 'اسفند 1402', 'اسفند ۱۴۰۲', 'بهار 1403', 'بهار ۱۴۰۳', 'بهار ۴۰۳', 'بهار 403', 'بهتر ۴۰۳', 'بهمن ۱۰۴۲', 'بهمن ۴۰۲ ۴۰۳', 'بهمن۴۰۲-۳', 'بهمن 1402', '1402 بهمن', '1402بهمن', 'بهمن  1402', 'بهمن 1402-1403', 'بهمن 402', 'بهمن ۱۴۰۲', 'بهمن ۴۰۲', 'بهمن۱۴۰۲', 'ترم دو ۱۴۰۲_۱۴۰۳', 'زمستان ۱۴۰۲', 'زمستان ۴۰۲', 'زمستان 402', 'خرداد ۴۰۳', 'زمستان ۱۴۰۲ و بهار ۱۴۰۳', 'زمستان ۴۰۳', '۱۴۰۲٫۱۴۰۳', '۱۴۰۲_۱۴۰۳', '۴۰۲.۲'): 'second_1402',
 ('تابستان 1402',): 'summer_1402',
 ('1402', '۱۴۰۲', 'اول و دوم ۱۴۰۲', 'مهر 1402 و بهمن 1402', 'مهر و بهمن ۴۰۲', '۱۴۰۲ ۱۴۰۳'): 'both_1402',
 
 ('1403 مهر', '403', '4031', 'مهر 1403', 'مهر 403', 'مهر ۱۴٠۳', 'مهر ۱۴۰۳', 'مهر ١۴٠٣', 'مهر ۴۰۳', 'مهر1403', 'مهر403', 'مهر۱۴۰۳', 'مهر۴۰۳', 'مهر۴۰۳- بهمن۴۰۳', 'نیمه اول ۱۴۰۳', 'پائیز 1403', 'پاییز 1403', 'پاییز 403', 'پاییز ۱۴۰۳', 'پاییز ۴۰۳', 'پاییز۴۰۳'): 'first_1403',
 ('1403-2', '4032', '403_2', 'بهار ۱۴۰۴', 'بهار ۴۰۴','بهمن 1403', 'بهمن 403', 'بهمن ١٤٠٣', 'بهمن ۱۴٠۳', 'بهمن ۱۴۰۳', 'بهمن ۴۰۳', 'بهمن۱۴۰۳', 'بهمن ۴۰۲-۳', 'بهمن۴۰۳', 'ترم زمستان 1403', '۴۰۳-۲', '۴۰۳۲'): 'second_1403',
 ('تابستان 1403', 'تابستان 403', 'تابستان ۱۴۰۳', 'تابستان ۴۰۳'): 'summer_1403',
 ('۱۴۰۳', '۴۰۳', 'آبان و بهمن ۱۴۰۳', 'بهمن ۱۴۰۳ و مهر ۱۴۰۳', 'مهر و بهمن 1403', 'مهر و بهمن ۴۰۳', 'مهر ۱۴۰۳ و بهمن ۴۰۳-۴۰۴', 'مهر۱۴۰۳و بهمن ۱۴۰۳'): 'both_1403',

 ('مهر 1404', '۴۰۴'): 'first_1404',
 ('بهمن 1404', 'بهمن ۱۴۰۴', 'بهمن ۴۰۴'): 'second_1404',
 ('تابستان 1404', 'تابستان ۴٠۴'): 'summer_1404'
}

flat_mapping = {
    key: value
    for keys, value in temp_grouped.items()
    for key in keys
}

In [8]:
df['term'] = (
    df['term']
      .astype(str)
      .str.strip()
      .str.replace(r'([آ-یA-Za-z]+)([0-9]+)', r'\1 \2', regex=True)
      .map(flat_mapping)
)

### Professor Names Normalization

In [9]:
# Checking For Common Prefixes
prefixes = ['سید', 'دکتر', 'مهندس', 'آقای', 'آقا', 'خانم', 'خانوم', 'سرکار', 'سرکار خانم', 'سرکار خانوم', 'استاد', 'پروفسور', 'پرفسور', 'جناب', 'جناب آقای']

pc = {p: 0 for p in prefixes}
for n in df['professor_name_raw']:
    s = str(n).strip()
    for p in prefixes:
        if s.startswith(p):
            pc[p] += 1
            break

for p, c in sorted(pc.items(), key=lambda x: -x[1]):
    if c: print(f"{p}: {c}")

دکتر: 153
سید: 143
استاد: 88
خانم: 2
پروفسور: 2
مهندس: 1
آقای: 1
آقا: 1
پرفسور: 1


In [10]:
PUNCT_RE = re.compile(r'[^\w\sآ-ی]')
VA_RE = re.compile(r'^\s*.+\s+و\s+.+\s*$')

mask = (
    df['professor_name_raw'].fillna('').str.contains(PUNCT_RE.pattern, na=False, regex=True) |
    df['professor_name_raw'].fillna('').str.contains(VA_RE.pattern, na=False, regex=True)
)
combined = df[mask]
combined['professor_name_raw']

324                                   🧑‍
1013              استاد سیدین و ابوطالبی
1946                    اکبری - محمودوند
1981           معصومه احمدی-زهرا عابدینی
2025                خانم احمدی و عابدینی
2066              دکتر زمانی-مهندس احمدی
2118                  نوروزبیگی-سعیدزاده
2445       استاد عابدینی، داوودی و اکبری
2605    استاد چوپانکاره و استاد میرشفیعی
2635          مجید نوروزی - امید محمدیان
2708        مریم صالحی و حمید رضا کوهدار
2890                بوترابی، عشیری، رضوی
3049           علیرضا معینی، صدیقه شریفی
3267       مهرانه تیراندری -زهرا عابدینی
3284                                  🧑‍
3401          دکتر سیدین و دکتر ابوطالبی
3632             گرشاسبی و زهرا بیرانوند
4590                     ساکیانی-اقاجانی
Name: professor_name_raw, dtype: object

In [11]:
inp = 'professor_name_raw'
out = 'prof_merged'

# INITIAL NORMALIZE
CHAR_MAP = {'ي':'ی','ى':'ی','ے':'ی','ك':'ک'}
ZW_RE = re.compile(r'[\u200c\u200d\uFEFF]')

def basic_normalize(s):
    if not isinstance(s, str):
        return ''
    s = unicodedata.normalize('NFKC', s.strip())
    s = ''.join(ch for ch in s if not unicodedata.combining(ch))
    for a, b in CHAR_MAP.items():
        s = s.replace(a, b)
    s = ZW_RE.sub('', s)
    return re.sub(r'\s+', ' ', s).strip()

# Prefixes Removal
PREFIXES = ['سیده', 'سید', 'مهندس', 'استاد', 'خانم', 'پروفسور', 'پرفسور', 'آقای', 'آقا', 'اقای دکتز']
pat_start = re.compile(r'^\s*(?:' + '|'.join(map(re.escape, PREFIXES)) + r')(?=\s|[آ-ی]|$)\s*')

# Special Case
SPECIAL_CASES = ['ریاضی دو', 'زبان عمومی']
pat_special = re.compile(r'\b(?:' + '|'.join(map(re.escape, SPECIAL_CASES)) + r')\b')

# Special Token Fix
FIXES = [
    (re.compile(r'\bاقا(?=[آ-ی])'), 'آقا'),
    (re.compile(r'\bاشتیانی\b'), 'آشتیانی'),
    (re.compile(r'\bازاد\b'), 'آزاد'),
    (re.compile(r'\bاسمعیلی\b'), 'اسماعیلی'),
    (re.compile(r'\bاللهی\b'), 'الهی'),
    (re.compile(r'الله'), 'اله'),
    (re.compile(r'الله'), 'اله'),
]


def clean_name(s):
    s = basic_normalize(s)

    has_doc = bool(re.search(r'\bدکتر\b', s))
    s = re.sub(r'\bدکتر\b', ' ', s)

    s = pat_special.sub(' ', s)

    while True:
        new = pat_start.sub('', s)
        if new == s:
            break
        s = new

    for p, r in FIXES:
        s = p.sub(r, s)

    s = re.sub(r'\s+', ' ', s).strip()
    return s, has_doc

def format_rep(base, has_doc):
    return ('دکتر ' + base) if (has_doc and base) else base

In [12]:
DEPT_TH = {'فیزیک': 0.75, 'مکانیک': 0.75, 'مهندسی_مواد': 0.80, 'مهندسی_کامپیوتر': 0.71,
'برق': 0.76, 'مهندسی_عمران': 0.71, 'معارف': 0.79, 'مهندسی_شیمی': 0.80, 'ریاضی': 0.72, 'مهندسی_راه_آهن': 0.79,}

def name_similarity(a, b, dept=None):
    a = re.sub(r'\s+', ' ', a.strip())
    b = re.sub(r'\s+', ' ', b.strip())
    if not a or not b:
        return False

    a_tokens = a.split()
    b_tokens = b.split()
    num_a = len(a_tokens)
    num_b = len(b_tokens)

    threshold = DEPT_TH.get(dept, 0.75)

    if num_a == 1 and num_b == 1:
        threshold = 0.85

    is_single_vs_last = False
    if min(num_a, num_b) == 1 and max(num_a, num_b) > 1:
        short_token = a_tokens[0] if num_a == 1 else b_tokens[0]
        long_tokens = b_tokens if num_a == 1 else a_tokens
        if short_token == long_tokens[-1]:
            is_single_vs_last = True

    a_no_space = ''.join(a_tokens)
    b_no_space = ''.join(b_tokens)
    similarity = difflib.SequenceMatcher(None, a_no_space, b_no_space).ratio()

    if is_single_vs_last:
        return True

    return similarity >= threshold

In [13]:
def merge_by_dept(df, inp, out):
    df[out] = ''

    for department, sub in df.groupby('department'):
        key_to_rows = defaultdict(list)

        for idx, orig in sub[inp].fillna('').items():
            base, has_doc = clean_name(orig)
            if base:
                key_to_rows[base].append((idx, has_doc))

        keys = list(key_to_rows)
        if not keys:
            continue

        G = nx.Graph()
        G.add_nodes_from(keys)

        for a, b in combinations(keys, 2):
            if name_similarity(a, b, department):
                G.add_edge(a, b)

        for cluster in nx.connected_components(G):
            rep_base = max(cluster, key=lambda k: (len(key_to_rows[k]), len(k.split())))
            doc_present = any(has_doc for k in cluster for _, has_doc in key_to_rows[k])
            rep = format_rep(rep_base, doc_present)

            for k in cluster:
                for idx, _ in key_to_rows[k]:
                    df.at[idx, out] = rep

    return df

In [14]:
manual_cases = {
    # فیزیک
    'راه چمنی': 'ذبیح اله راه چمنی', 'راهچمنی': 'ذبیح اله راه چمنی',
    'زهرا عابدینی امین آباد': 'زهرا عابدینی', 
    'زهرا کی احمدی': 'زهرا کی احمدی',

    # مکانیک
    'آیت الهی': 'مجیدرضا آیت الهی',

    # مهندسی مواد
    'رضوی عشیری': 'روح الهی عشیری',

    # مهندسی کامپیوتر
    'زهرا رشیدی': 'زهرا رشیدی', 'رضا انتظار ملکی': 'رضا انتظاری ملکی',

    # برق
    'داوود عرب': 'داوود عرب خابوری', 'عرب': 'داوود عرب خابوری',
    'حسین عرب': 'حسین عرب',
    'رحیمی پور': 'رحیمی پور',
    'راحمی': 'راحمی نوش آبادی',
    'حاجی احمدی': 'محمد جواد حاجی احمدی',
    'توتونچیان': 'فرید توتونچیان',

    # مهندسی عمران
    'علیزاده': 'حسین علیزاده', 'حسین علیزاده': 'حسین علیزاده',
    'علی کاوه': 'علی کاوه',

    # معارف
    'رهبری': 'سعید رهبری شمیرانی', 'سعید رهبری': 'سعید رهبری شمیرانی',
    'هوشمند': 'دکتر نرگس هوشمند همدانی', 'نرگس هوشمند': 'دکتر نرگس هوشمند همدانی', 'دکتر نرگس هوشمند': 'دکتر نرگس هوشمند همدانی',
    'حسن محسنی': 'حسن محسنی',
    'شاه علی': 'احمدرضا شاه علی',
    'تاج الدین': 'علی محمد تاج الدین',
    'هادی نوراله': 'نوراله هادی',
    'محمد فارسانی امینی': 'محمد امینی فارسانی',

    # مهندسی شیمی
    'احمد رهبر': 'احمد رهبر کلیشمی', 'رهبر': 'احمد رهبر کلیشمی',

    # ریاضی
    'محمدهادی علاییان': 'محمد هادی علائیان', 'محمدهادی علائیان': 'محمد هادی علائیان',  'محمد هادی علائیان': 'محمد هادی علائیان',
    'جانی': 'اسداله آقاجانی',
    'تومانیان مگردیچ': 'مگردیچ تومانیان',

    # مهندسی راه آهن
    'حمید رضا احدی': 'حمیدرضا احدی', 'حمیدرضا احدی': 'حمیدرضا احدی',
    'حمیدرضا حیدری': 'حمیدرضا حیدری', 'حمیدرضا حیدری نوقابی': 'حمیدرضا حیدری',
    'امین اوحدی اصفهانی': 'امین اوحدی',
}

In [15]:
def print_clusters_by_department(df, inp, out, dept):
    sub = df[df['department'] == dept]
    if sub.empty:
        print(f"No data for department: {dept}")
        return
    print(f"Department: {dept}")
    cluster_to_members = defaultdict(list)
    for _, row in sub.iterrows():
        base, _ = clean_name(row[inp])
        if base:
            cluster_to_members[row[out]].append(base)
    for rep, members in sorted(cluster_to_members.items()):
        unique_members = sorted(set(members))
        print(f"{unique_members} : {rep}")

In [16]:
dfn = df[~mask].copy()
dfn = merge_by_dept(dfn, inp, out)
inv = {v: k for k, v in manual_cases.items()}
dfn[out] = dfn['professor_name_raw'].map(inv).fillna(dfn[out])

In [17]:
dfn['department'].unique()

array(['برق', 'مهندسی_کامپیوتر', 'مهندسی_مواد', 'مکانیک', 'فیزیک',
       'ریاضی', 'مهندسی_صنایع', 'مهندسی_شیمی', 'معارف', 'مهندسی_عمران',
       'تربیت_بدنی', 'مهندسی_راه_آهن', 'معماری_و_شهرسازی', 'طراحی_صنعتی',
       'مهندسی_خودرو', 'مهندسی_پیشرفت', 'فناوری_های_نوین'], dtype=object)

In [18]:
print_clusters_by_department(dfn, inp, out, 'ریاضی')

Department: ریاضی
['ابراهیم اردشیر لاریجانی', 'اردشیر لاریجانی'] : ابراهیم اردشیر لاریجانی
['احمد رضا فروغ', 'احمد فروغ', 'احمدرضا فروغ', 'فروغ'] : احمدرضا فروغ
['ادریس فیض آبادی'] : ادریس فیض آبادی
['آقاجانی', 'اسداله آقاجانی'] : اسداله آقاجانی
['افشین نمیرانیان'] : افشین نمیرانیان
['اکبر دهقان نژاد', 'دهقان نژاد'] : اکبر دهقان نژاد
['تورج نیک آزاد', 'نیک آزاد'] : تورج نیک آزاد
['تومانیان مگردیچ', 'مگردیچ تومانیان'] : تومانیان مگردیچ
['اسداله آقاجانی', 'جانی'] : جانی
['جلیل رشیدی نیا', 'جلیل رشیدینیا', 'رشیدی نیا'] : جلیل رشیدی نیا
['حسن خرازی', 'خرازی'] : حسن خرازی
['حسینی پور'] : حسینی پور
['جوادی', 'حاجی سید جوادی', 'حمید حاجی سید جوادی'] : حمید حاجی سید جوادی
['حمید مسگرانی', 'مسگرانی'] : حمید مسگرانی
['حمیدرضا نوش کاران', 'حمیدرضا نوشکاران', 'نوشکاران'] : حمیدرضا نوشکاران
['حنیف حیدری'] : حنیف حیدری
['الهام اله مرادی'] : دکتر الهام اله مرادی
['الهام فتاحی', 'فتاحی'] : دکتر الهام فتاحی
['امید نیکان', 'مجیدیان'] : دکتر امید نیکان
['جواد وحیدی'] : دکتر جواد وحیدی
['رضا احمدی'] : دکت

In [19]:
dfn.loc[(dfn['department'] == 'ریاضی') & (dfn[inp] != dfn[out]), [inp, out, 'department']].drop_duplicates()

,professor_name_raw,prof_merged,department
41,الهام فتاحی,دکتر الهام فتاحی,ریاضی
42,رضا سعادتی,دکتر رضا سعادتی,ریاضی
43,احمد فروغ,احمدرضا فروغ,ریاضی
51,سمیه سعیدی نژاد,دکتر سمیه سعیدی نژاد,ریاضی
53,اسداله آقاجانی,جانی,ریاضی
...,...,...,...
4361,نوشکاران,حمیدرضا نوشکاران,ریاضی
4364,سيدحميدرضا نوش کاران,حمیدرضا نوشکاران,ریاضی
4417,سید حمیدرضا نوشکاران,حمیدرضا نوشکاران,ریاضی
4433,دکتر سعادتی,دکتر رضا سعادتی,ریاضی


In [20]:
# Final Normalization
def align_merge(df, merged='prof_merged', out1='dept', out2='professor'):
    df = df.copy()
    parsed = df[merged].map(clean_name)
    df['_base'] = parsed.map(lambda x: x[0])
    df['_has_doc'] = parsed.map(lambda x: x[1])

    df['_key'] = df['_base'].map(lambda x: ''.join(x.split()) if x else '')

    key_to_dept = (
        df.groupby('_key')['department']
        .apply(lambda x: Counter(x).most_common(1)[0][0])
        .to_dict()
    )

    key_has_doc = df.groupby('_key')['_has_doc'].any().to_dict()

    key_to_base = (
        df.groupby('_key')['_base']
        .apply(lambda x: Counter(x).most_common(1)[0][0])
        .to_dict()
    )

    df[out1] = df['_key'].map(key_to_dept)
    df[out2] = df['_key'].map(lambda k: format_rep(key_to_base[k], key_has_doc[k]))

    return df.drop(columns=['_base', '_has_doc', '_key'])

In [21]:
dfnn = align_merge(dfn)

In [22]:
changed = dfnn[dfnn['department'] != dfnn['dept']].drop_duplicates(subset=['prof_merged'])
changed[['professor_name_raw', 'prof_merged', 'professor', 'department', 'dept']]

,professor_name_raw,prof_merged,professor,department,dept
127,سعیدرضا مساح,سعیدرضا مساح,سعیدرضا مساح,مهندسی_راه_آهن,مهندسی_عمران
189,محمد شهبازی,محمد شهبازی,محمد شهبازی,مکانیک,معارف
192,وحید خطیبی,وحید خطیبی,وحید خطیبی,مهندسی_کامپیوتر,مهندسی_صنایع
208,سمیه سعیدی نژاد,سمیه سعیدی نژاد,دکتر سمیه سعیدی نژاد,مهندسی_عمران,ریاضی
256,هادی نوراله,هادی نوراله,هادی نوراله,برق,معارف
258,جواد غفاری,جواد غفاری,جواد غفاری,مهندسی_راه_آهن,معارف
283,رضا انتظار ملکی,رضا انتظار ملکی,رضا انتظار ملکی,برق,مهندسی_کامپیوتر
312,صادق رستم نیا,صادق رستم نیا,صادق رستم نیا,مهندسی_مواد,مهندسی_شیمی
333,محمد امینی فارسانی,محمد فارسانی امینی,محمد فارسانی امینی,مکانیک,معارف
368,محمود میرطباطبایی,محمود میرطباطبایی,محمود میرطباطبایی,مهندسی_عمران,معارف
